In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import stockdata as sd
import pandas as pd
import numpy as np

In [50]:
sql = '''
select * 
from histprice
where 1 = 1
and freq in ('d', 'w', 'm')
and (symbol like '%ETF%' or symbol like '%100%' or symbol like '%BEES%')
'''
df = sd.getdata(sql)

In [299]:
df.shape
df.head()
df[df.symbol == 'NIFTYBEES'][['symbol', 'freq']].value_counts()
df[df.symbol == 'BANKBEES'][['symbol', 'freq']].value_counts()
df[df.symbol == 'JUNIORBEES'][['symbol', 'freq']].value_counts()
df[df.symbol == 'GOLDBEES'][['symbol', 'freq']].value_counts()
df[df.symbol == 'NETFIT'][['symbol', 'freq']].value_counts()
df[df.symbol == 'N100'][['symbol', 'freq']].value_counts()
freq    = list(df.freq.unique())
symbols = list(df.symbol.unique())
freq
symbols

(3792, 16)

,freq,date,symbol,open,low,high,close,adjclose,volume,year,month,day,wkday,wknr,qrtr,runts
0,w,2008-12-29,NIFTYBEES,29.90,29.76,31.37,31.17,31.17,4656630,2008,12,29,1,1,4,Tue Feb-23-2021 11:00
1,w,2009-01-05,NIFTYBEES,31.10,27.56,31.50,27.73,27.73,10809220,2009,1,5,1,2,1,Tue Feb-23-2021 11:00
2,w,2009-01-12,NIFTYBEES,27.70,27.30,28.80,28.45,28.45,5327900,2009,1,12,1,3,1,Tue Feb-23-2021 11:00
3,w,2009-01-19,NIFTYBEES,28.39,27.04,28.55,27.21,27.21,3815280,2009,1,19,1,4,1,Tue Feb-23-2021 11:00
4,w,2009-01-26,NIFTYBEES,27.30,27.25,29.10,28.17,28.17,4828900,2009,1,26,1,5,1,Tue Feb-23-2021 11:00


symbol     freq
NIFTYBEES  w       635
           m       147
dtype: int64

symbol    freq
BANKBEES  w       635
          m       147
dtype: int64

symbol      freq
JUNIORBEES  w       635
            m       147
dtype: int64

symbol    freq
GOLDBEES  w       635
          m       147
dtype: int64

symbol  freq
NETFIT  w       23
        m        6
dtype: int64

symbol  freq
N100    w       516
        m       119
dtype: int64

['w', 'm']

['NIFTYBEES', 'BANKBEES', 'GOLDBEES', 'NETFIT', 'N100', 'JUNIORBEES']

**CAGR Calculation**

In [66]:
def CalculateCAGR(start, end, periods):
    return np.round((((end/start)**(1/periods)) - 1)*100, 2)

def CAGR(df, symbol, startyr, endyr):
    df = df[(df.symbol == symbol) & (df.freq == 'd') & (df.year >= startyr) & (df.year <= endyr)]
    start_price = df.open.iloc[0]
    final_price = df.close.iloc[-1]
    periods = endyr - startyr + 1
    cagr = CalculateCAGR(start_price, final_price, periods)
    return cagr

In [67]:
CAGR(df, 'NIFTYBEES', 2016, 2020)

13.2

In [73]:
freq    = 'd'
startyr = 2016
endyr   = 2020
period  = 5
df1 = df[(df.freq == freq) & (df.year >= startyr) & (df.year <= endyr)]
df1 = df1.groupby('symbol').agg({'open' : 'first', 'close' : 'last'}).reset_index()
df1['CAGR'] = df1.apply(lambda x: CalculateCAGR(x['open'], x['close'], period), axis = 1)
df1

,symbol,open,close,CAGR
0,BANKBEES,170.20,313.54,13.00
1,GOLDBEES,22.85,43.75,13.87
2,JUNIORBEES,201.00,340.27,11.10
3,N100,305.99,926.68,24.81
4,NETFIT,20.98,24.54,3.18
5,NIFTYBEES,80.20,149.07,13.20


In [69]:
symbols = ['NIFTYBEES', 'BANKBEES', 'JUNIORBEES', 'GOLDBEES', 'NETFIT', 'N100']
cagr_5y  = [CAGR(df, s, 2016, 2020) for s in symbols]
cagr_10y = [CAGR(df, s, 2011, 2020) for s in symbols]
x = pd.DataFrame({'symbols':symbols, 'cagr_5y':cagr_5y, 'cagr_10y':cagr_10y})
x
# x.to_csv('ETS_CAGR.csv')

,symbols,cagr_5y,cagr_10y
0,NIFTYBEES,13.20,9.15
1,BANKBEES,13.00,10.10
2,JUNIORBEES,11.10,10.71
3,GOLDBEES,13.87,8.04
4,NETFIT,3.18,1.58
5,N100,24.81,24.33


**ROI Calculation**

In [74]:
df.head()

,freq,date,symbol,open,low,high,close,adjclose,volume,year,month,day,wkday,wknr,qrtr,runts
0,d,2009-01-02,NIFTYBEES,30.60,30.30,30.94,30.56,30.56,1344830,2009,1,2,5,1,1,Tue Feb-23-2021 10:59
1,d,2009-01-05,NIFTYBEES,30.70,30.70,31.37,31.17,31.17,2429180,2009,1,5,1,2,1,Tue Feb-23-2021 10:59
2,d,2009-01-06,NIFTYBEES,31.10,30.60,31.48,31.12,31.12,1418730,2009,1,6,2,2,1,Tue Feb-23-2021 10:59
3,d,2009-01-07,NIFTYBEES,31.38,28.90,31.50,29.26,29.26,5724640,2009,1,7,3,2,1,Tue Feb-23-2021 10:59
4,d,2009-01-09,NIFTYBEES,29.00,28.03,29.50,28.85,28.85,1820800,2009,1,9,5,2,1,Tue Feb-23-2021 10:59


In [76]:
def calculateROI(df, freq, startyr, endyr):
    df= df[(df.freq == freq) & (df.year >= startyr) & (df.year <= endyr)]
    x = df.groupby('symbol').agg({'close' : ['max', 'mean', 'count']})
    y = df.groupby('symbol').agg({'year' : ['min' , 'max']})
    z = pd.merge(x, y, on="symbol").reset_index()
    period = endyr - startyr + 1
    z.columns = ['symbol', 'ltp', 'avg price', 'qty', 'from', 'to']
    z['period'] = z.apply(lambda x: f"{x['from']}-{x['to']}", axis = 1)
    z.drop(['from', 'to'], axis = 1, inplace = True)
    z['invested'] = z.apply(lambda x: x['avg price'] * x['qty'], axis = 1)
    z[f'{period}y_{freq}_roi']  = z.apply(lambda x: x['ltp'] * x['qty'], axis = 1)
    z[f'{period}y_{freq}_roi%'] = z.apply(lambda x: (x[f'{period}y_{freq}_roi']/x['invested']), axis = 1)
    return z

In [78]:
calculateROI(df, 'w', 2016, 2020)
calculateROI(df, 'm', 2016, 2020)

,symbol,ltp,avg price,qty,period,invested,5y_w_roi,5y_w_roi%
0,BANKBEES,330.74,244.015479,261,2016-2020,63688.04,86323.14,1.355406
1,GOLDBEES,48.40,30.685900,261,2016-2020,8009.02,12632.40,1.577272
2,JUNIORBEES,342.19,267.641111,261,2016-2020,69854.33,89311.59,1.278540
3,N100,931.17,507.767280,261,2016-2020,132527.26,243035.37,1.833852
4,NETFIT,24.70,22.050000,15,2020-2020,330.75,370.50,1.120181
5,NIFTYBEES,150.71,106.450230,261,2016-2020,27783.51,39335.31,1.415779


,symbol,ltp,avg price,qty,period,invested,5y_m_roi,5y_m_roi%
0,BANKBEES,329.00,248.971333,60,2016-2020,14938.28,19740.0,1.321437
1,GOLDBEES,47.14,31.035000,60,2016-2020,1862.10,2828.4,1.518930
2,JUNIORBEES,340.27,271.096833,60,2016-2020,16265.81,20416.2,1.255160
3,N100,938.46,520.147667,60,2016-2020,31208.86,56307.6,1.804218
4,NETFIT,24.95,23.162500,4,2020-2020,92.65,99.8,1.077172
5,NIFTYBEES,149.07,108.142000,60,2016-2020,6488.52,8944.2,1.378465


In [475]:
wroi_1y  = calculateROI(df, 'w', 2020, 2020)
wroi_2y  = calculateROI(df, 'w', 2019, 2020)
wroi_3y  = calculateROI(df, 'w', 2018, 2020)
wroi_5y  = calculateROI(df, 'w', 2016, 2020)
wroi_10y = calculateROI(df, 'w', 2011, 2020)
mroi_1y  = calculateROI(df, 'm', 2020, 2020)
mroi_2y  = calculateROI(df, 'm', 2019, 2020)
mroi_3y  = calculateROI(df, 'm', 2018, 2020)
mroi_5y  = calculateROI(df, 'm', 2016, 2020)
mroi_10y = calculateROI(df, 'm', 2011, 2020)
x = pd.concat([wroi_1y, wroi_2y, wroi_3y, wroi_5y, wroi_10y, mroi_1y, mroi_2y, mroi_3y, mroi_5y, mroi_10y], ignore_index=True)

In [477]:
x.to_csv('roi.csv')